# 自动求导
- 假设我们想对函数$y = 2x^Tx$关于列向量$x$进行求导

In [2]:
import torch

In [3]:
x = torch.arange(4.0)

x

tensor([0., 1., 2., 3.])

  - 在计算$y$关于$\vec{x}$的梯度之前，我们需要告诉python，用一个地方来存储梯度

In [4]:
x.requires_grad_(True)
x.grad

- 通过调用反向传播来自动计算y关于x每个分量的梯度

In [5]:
# 现在来计算y
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [6]:
y.backward()
x.grad


tensor([ 0.,  4.,  8., 12.])

In [7]:
x.grad == 4 * x

tensor([True, True, True, True])

In [8]:
# z = 2 * x
# z 
# x.grad.zero_()
# z.backward()
# x.grad

In [9]:
# 在默认情况下，PyTorch会累计梯度，我们需要清除之前计算的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [ ]:
# 将梯度清零
x.grad.zero_()
y = x * x
# 将y张量从计算图中分离出来，返回一个新的张量u
# 这个张量与y相同
# 但是不具有梯度信息
# 将u看做一个标量
u = y.detach()

z = u * x
# z.sum()是计算z张量所有元素的和
# .backward()是计算z相对于输入张量的梯度（求偏导），并将结果存储在对应的张量的.grad属性中
# 由于u是一个标量，所以z.sum().backward就是z对x求导
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [11]:
x.grad.zero_()
y.sum().backward()

x.grad == 2 * x

tensor([True, True, True, True])

In [ ]:
x.grad.zero_()
y = 2 * x
# .backward()函数只有用标量输出
# 所以必须保证.前面的内容是一个标量
y.sum().backward()

x.grad


tensor([2., 2., 2., 2.])

- 对于f(a)
- 首先将$a$输入到函数中，此时：$$b = 2 * a$$
- 接下来进行$k$次操作，直至b的大小大于等于1000，此时：$$b = 2 * a * 2^k = 2^{k + 1} * a$$
- 接下俩对$b$进行操作，分为乘1或者乘100，此时：$$c = t * 2^{k + 1} * a \quad (t =1,100)$$
- 返回$c$值，此时$$d = c$$
- $d$对$a$进行求导，可以发现$$\text{grad}(d) = t * 2^{k + 1} = d / a$$

In [19]:
def f(a):
  b = a * 2
  while b.norm() < 1000:
    b = b * 2
  if b.sum() > 0:
    c = b
  else:
    c = 100 * b
  return c

a = torch.randn(size=(),requires_grad=True)
a
d = f(a)
d.backward()

a.grad == d / a


tensor(True)